In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from cim_optimizer.solve_Ising import Ising

# 1. 生成 MAX-CUT 图
n_nodes = 5  # 节点数
np.random.seed(42)  # 固定随机种子
G = nx.Graph()
G.add_nodes_from(range(n_nodes))

# 随机生成边（约 50% 概率）
for i in range(n_nodes):
    for j in range(i + 1, n_nodes):
        if np.random.rand() > 0.5:
            G.add_edge(i, j, weight=1.0)  # 边权重为 1

# 绘制图
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=16)
nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): f"{d['weight']}" for u, v, d in G.edges(data=True)})
plt.savefig("max_cut_graph.png")
plt.close()
print("图已生成并保存为 max_cut_graph.png")
print("边:", list(G.edges()))


图已生成并保存为 max_cut_graph.png
边: [(0, 2), (0, 3), (0, 4), (2, 3), (2, 4), (3, 4)]


In [3]:

# 2. 构建 QUBO 矩阵
N = n_nodes  # 变量数 = 节点数
Q = np.zeros((N, N))

# MAX-CUT QUBO 公式：min sum_{(i,j) in E} (x_i + x_j - 2 x_i x_j)
for i, j in G.edges():
    Q[i, i] += 1  # x_i 项
    Q[j, j] += 1  # x_j 项
    Q[i, j] += -2  # -2 x_i x_j
    Q[j, i] += -2  # 对称

# 验证 QUBO 矩阵
print(f"QUBO 矩阵形状: {Q.shape}")
print(f"QUBO 矩阵是否对称: {np.allclose(Q, Q.T)}")
print("QUBO 矩阵:\n", Q)

# 保存 QUBO 矩阵到 CSV
labels = [f"v_{i}" for i in range(n_nodes)]
Q_df = pd.DataFrame(Q, index=labels, columns=labels)
Q_df.to_csv("max_cut_qubo.csv")
print("QUBO 矩阵已保存为 max_cut_qubo.csv")

QUBO 矩阵形状: (5, 5)
QUBO 矩阵是否对称: True
QUBO 矩阵:
 [[ 3.  0. -2. -2. -2.]
 [ 0.  0.  0.  0.  0.]
 [-2.  0.  3. -2. -2.]
 [-2.  0. -2.  3. -2.]
 [-2.  0. -2. -2.  3.]]
QUBO 矩阵已保存为 max_cut_qubo.csv


In [4]:
# 3. 转换为伊辛模型
J = np.zeros((N, N))
h = np.zeros(N)
for i in range(N):
    h[i] = Q[i, i] / 4 + sum(Q[i, j] / 4 for j in range(N) if j != i)
    for j in range(i + 1, N):
        J[i, j] = Q[i, j] / 4
J = J + J.T
h += np.sum(Q, axis=1) / 4


In [5]:
# 4. 使用 CIM 求解
try:
    test = Ising(J, h).solve(
        num_runs=50,
        num_timesteps_per_run=5000,
        cac_time_step=0.02,
        cac_r=-0.5,
        cac_alpha=0.8,
        cac_beta=0.3,
        cac_gamma=0.005,
        cac_delta=15,
        cac_mu=0.9,
        cac_rho=1.5,
        cac_tau=200,
        return_spin_trajectories_all_runs=True,
        hyperparameters_randomtune=False
    )

    # 提取结果
    print("test.result 属性:", [attr for attr in dir(test.result) if not attr.startswith('_')])
    best_energy = test.result.lowest_energy
    print(f"最佳能量: {best_energy}")
    best_spins = test.result.lowest_energy_spin_config

    # 转换为 QUBO 变量（0,1）
    x_sol = [(best_spins[i] + 1) / 2 for i in range(N)]
    print(f"切割分配: {x_sol}")

    # 计算切割值
    cut_value = 0
    for i, j in G.edges():
        if x_sol[i] != x_sol[j]:
            cut_value += 1  # 边权重为 1
    print(f"切割值（跨组边数）: {cut_value}")

    # 验证切割
    set_0 = [i for i, x in enumerate(x_sol) if x == 0]
    set_1 = [i for i, x in enumerate(x_sol) if x == 1]
    print(f"集合 S: {set_0}")
    print(f"集合 T: {set_1}")

    # 可视化切割结果
    node_colors = ['red' if x_sol[i] == 0 else 'green' for i in range(n_nodes)]
    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_size=16)
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): f"{d['weight']}" for u, v, d in G.edges(data=True)})
    plt.savefig("max_cut_solution.png")
    plt.close()
    print("切割结果已保存为 max_cut_solution.png")

except Exception as e:
    print(f"CIM 求解失败: {e}")

External Field Detected
Target Ising Energy: -inf.
Best Ising Energy Found: -3.0.
Corresponding Spin Configuration: [-1. -1. -1. -1. -1.].
Time Elapsed: 144.41263580322266.
Number of Runs Completed: 50.
test.result 属性: ['energies', 'energy_evolution', 'lowest_energy', 'lowest_energy_spin_config', 'result_data', 'spin_config_all_runs', 'spin_trajectories', 'time']
最佳能量: -3.0
切割分配: [0.0, 0.0, 0.0, 0.0, 0.0]
切割值（跨组边数）: 0
集合 S: [0, 1, 2, 3, 4]
集合 T: []
切割结果已保存为 max_cut_solution.png
